# WeatherBench2 Evaluation for Flow Matching Models

This notebook demonstrates how to evaluate weather prediction models using the WeatherBench2 benchmark. WeatherBench2 is a comprehensive benchmark for data-driven weather forecasting that allows for fair comparison of different approaches.

We'll cover:
1. Setting up the WeatherBench2 evaluation framework
2. Loading trained flow matching models
3. Generating predictions on the test dataset
4. Computing standard evaluation metrics
5. Comparing with baselines and state-of-the-art models
6. Creating visualization dashboards for model performance

This allows researchers to understand how well their flow matching models perform compared to other approaches in the field.

## 1. Setup and Dependencies

In [ ]:
# Add repository root to Python path to find weatherflow package
import sys
import os
# Get absolute path to repo root
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
repo_root = os.path.abspath(os.path.join(notebook_dir, '..'))
# Add to path if not already there
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
print(f"Added {repo_root} to Python path")

# Add repository root to Python path
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname('__file__'), '..')))

# Install WeatherFlow if needed
try:
    import weatherflow
    print(f"WeatherFlow version: {weatherflow.__version__}")
except ImportError:
    !pip install -e ..
    import weatherflow
    print(f"WeatherFlow installed, version: {weatherflow.__version__}")

# Install WeatherBench2 (if not already included)
try:
    import weatherbench2
    print(f"WeatherBench2 version: {weatherbench2.__version__}")
except ImportError:
    !pip install git+https://github.com/google-research/weatherbench2.git
    import weatherbench2
    print(f"WeatherBench2 installed, version: {weatherbench2.__version__}")

# Import standard libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import torch
from tqdm.notebook import tqdm
import os
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')  # Suppress some warnings for cleaner output

# Import WeatherFlow components
from weatherflow.data import ERA5Dataset, create_data_loaders
from weatherflow.models import WeatherFlowMatch, WeatherFlowODE
from weatherflow.utils import WeatherVisualizer, WeatherMetrics

# Import WeatherBench2 components
from weatherbench2 import metrics
from weatherbench2 import evaluation

# Set up matplotlib
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['figure.dpi'] = 100

# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set up directories
os.makedirs("../evaluations", exist_ok=True)
eval_dir = "../evaluations/weatherbench2"
os.makedirs(eval_dir, exist_ok=True)
plot_dir = os.path.join(eval_dir, "plots")
os.makedirs(plot_dir, exist_ok=True)

print(f"Evaluation results will be saved to: {eval_dir}")
print(f"Plots will be saved to: {plot_dir}")

In [ ]:
# Mock dependencies that might not be availabletry:    import sys    notebook_dir = os.path.dirname(os.path.abspath('__file__'))    repo_root = os.path.abspath(os.path.join(notebook_dir, '..'))    mock_path = os.path.join(repo_root, 'mock_dependencies.py')        if os.path.exists(mock_path):        # Execute the mock dependencies script        with open(mock_path, 'r') as f:            mock_code = f.read()            # Add repo_root to sys.path if not already there            if repo_root not in sys.path:                sys.path.insert(0, repo_root)            # Execute the script            exec(mock_code)            # Call the function to install all mocks            exec("install_all_mocks()")    else:        print(f"Warning: Mock dependencies script not found at {mock_path}")except Exception as e:    print(f"Error loading mock dependencies: {str(e)}")

## 2. Configuration

In [ ]:
# Set up configuration for the evaluation
config = {
    # Model configuration
    "model_path": "../models/flow_match_20250226_123456_best.pt",  # Update with your model path
    "use_pretrained": True,  # Set to False to use your own model
    "pretrained_model": "era5_z500_2016_2017",  # Name of pretrained model to use
    
    # Data configuration
    "variables": ["z", "t", "u", "v"],  # Variables to evaluate
    "pressure_levels": [500, 850],  # Pressure levels to evaluate
    "test_period": ("2018-01-01", "2018-12-31"),  # Test period
    "data_dir": None,  # Set to None to use WeatherBench2 default data
    
    # Evaluation configuration
    "lead_times": [6, 24, 72, 120, 168],  # Lead times in hours to evaluate
    "climatology_period": ("2015-01-01", "2017-12-31"),  # Period for climatology baseline
    "n_samples": 50,  # Number of samples to evaluate (smaller for demonstration)
    "batch_size": 16,  # Batch size for prediction
    
    # Baseline models to compare with
    "compare_baselines": True,  # Compare with WeatherBench2 baselines
    "baselines": ["persistence", "climatology", "ifs", "fourcastnet"],  # Baselines to include
    
    # Visualization settings
    "create_plots": True,
    "plot_variables": ["z500", "t850"],  # Variables to plot results for
    "plot_metrics": ["rmse", "acc"],  # Metrics to plot
}

# Display configuration
print("\nEvaluation Configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")

## 3. Load Model

In [ ]:
# Function to load a pre-trained model
def load_model(config):
    """Load a WeatherFlowMatch model for evaluation.
    
    Args:
        config: Configuration dictionary
        
    Returns:
        model: Loaded model
        model_info: Model information
    """
    if config["use_pretrained"]:
        # Load a pretrained model
        pretrained_dir = "../models/pretrained"
        model_path = os.path.join(pretrained_dir, f"{config['pretrained_model']}.pt")
        
        # Check if pretrained model exists
        if not os.path.exists(model_path):
            print(f"Pretrained model not found at {model_path}")
            print("Creating a dummy model for demonstration purposes.")
            
            # Create a dummy model
            model = WeatherFlowMatch(
                input_channels=len(config["variables"]),
                hidden_dim=64,
                n_layers=3,
                use_attention=True,
                physics_informed=True
            )
            model_info = {
                "variables": config["variables"],
                "pressure_levels": config["pressure_levels"],
                "config": {
                    "hidden_dim": 64,
                    "n_layers": 3,
                    "use_attention": True,
                    "physics_informed": True
                }
            }
            return model.to(device), model_info
    else:
        # Load custom model
        model_path = config["model_path"]
    
    # Load the model checkpoint
    try:
        checkpoint = torch.load(model_path, map_location=device)
        
        # Extract model configuration
        if "config" in checkpoint:
            model_info = checkpoint["config"]
        else:
            # Default configuration if not found
            model_info = {
                "hidden_dim": 128,
                "n_layers": 4,
                "use_attention": True,
                "physics_informed": True,
                "variables": config["variables"],
                "pressure_levels": config["pressure_levels"]
            }
        
        # Create model with the same architecture
        model = WeatherFlowMatch(
            input_channels=len(config["variables"]),
            hidden_dim=model_info.get("hidden_dim", 128),
            n_layers=model_info.get("n_layers", 4),
            use_attention=model_info.get("use_attention", True),
            physics_informed=model_info.get("physics_informed", True)
        )
        
        # Load weights
        model.load_state_dict(checkpoint["model_state_dict"])
        print(f"Successfully loaded model from {model_path}")
        
        return model.to(device), model_info
    
    except Exception as e:
        print(f"Error loading model: {str(e)}")
        print("Creating a dummy model for demonstration purposes.")
        
        # Create a dummy model
        model = WeatherFlowMatch(
            input_channels=len(config["variables"]),
            hidden_dim=64,
            n_layers=3,
            use_attention=True,
            physics_informed=True
        )
        model_info = {
            "variables": config["variables"],
            "pressure_levels": config["pressure_levels"],
            "config": {
                "hidden_dim": 64,
                "n_layers": 3,
                "use_attention": True,
                "physics_informed": True
            }
        }
        return model.to(device), model_info

# Load the model
print("\nLoading model...")
model, model_info = load_model(config)
model.eval()

# Print model information
print("\nModel Information:")
print(f"  Variables: {model_info.get('variables', config['variables'])}")
print(f"  Pressure Levels: {model_info.get('pressure_levels', config['pressure_levels'])}")
print(f"  Hidden Dimension: {model_info.get('hidden_dim', 128)}")
print(f"  Number of Layers: {model_info.get('n_layers', 4)}")
print(f"  Using Attention: {model_info.get('use_attention', True)}")
print(f"  Physics Informed: {model_info.get('physics_informed', True)}")

## 4. Set up WeatherBench2 Evaluation

In [ ]:
# WeatherBench2 uses specific variable names and datasets
# We need to convert between WeatherFlow and WeatherBench2 formats

# Define variable mapping between WeatherFlow and WeatherBench2
var_mapping = {
    "z": "geopotential",
    "t": "temperature",
    "u": "u_component_of_wind",
    "v": "v_component_of_wind",
    "q": "specific_humidity",
    "r": "relative_humidity"
}

# Define pressure level mapping for combined variable names
level_vars = {
    "z": {500: "z500", 850: "z850", 250: "z250"},
    "t": {500: "t500", 850: "t850", 250: "t250"},
    "u": {500: "u500", 850: "u850", 250: "u250"},
    "v": {500: "v500", 850: "v850", 250: "v250"},
    "q": {500: "q500", 850: "q850", 250: "q250"},
    "r": {500: "r500", 850: "r850", 250: "r250"}
}

# Function to load WeatherBench2 data
def load_wb2_data(variables, pressure_levels, time_period, data_dir=None):
    """Load data from WeatherBench2 format.
    
    Args:
        variables: List of variables to load
        pressure_levels: List of pressure levels
        time_period: Tuple of (start_date, end_date)
        data_dir: Optional directory for data
        
    Returns:
        Dictionary of xarray datasets
    """
    # This is a simplified version - in a full implementation,
    # we would use the WeatherBench2 API to load the data
    
    print(f"Loading WeatherBench2 data for {time_period}...")
    
    # For demonstration, we'll create a dummy dataset with the right structure
    # In a real implementation, we would load actual WeatherBench2 data
    
    # Create time range
    time = pd.date_range(time_period[0], time_period[1], freq="6H")
    
    # Create latitude and longitude
    lat = np.linspace(-90, 90, 32)
    lon = np.linspace(0, 360, 64, endpoint=False)
    
    # Create datasets for each variable and level
    datasets = {}
    
    for var in variables:
        wb2_var = var_mapping.get(var, var)
        
        for level in pressure_levels:
            # Create variable name
            level_var = level_vars.get(var, {}).get(level, f"{var}{level}")
            
            # Create dummy data array
            data = np.random.randn(len(time), len(lat), len(lon))
            
            # Create dataset
            ds = xr.Dataset(
                data_vars={
                    wb2_var: (["time", "latitude", "longitude"], data)
                },
                coords={
                    "time": time,
                    "latitude": lat,
                    "longitude": lon,
                    "level": [level]
                }
            )
            
            datasets[level_var] = ds
    
    print(f"Loaded {len(datasets)} datasets.")
    return datasets

# Load test data
print("\nLoading test data...")
test_data = load_wb2_data(
    variables=config["variables"],
    pressure_levels=config["pressure_levels"],
    time_period=config["test_period"],
    data_dir=config["data_dir"]
)

In [ ]:
# Function to generate predictions using the WeatherFlow model
def generate_predictions(model, test_data, lead_times, n_samples, batch_size):
    """Generate predictions using the WeatherFlow model.
    
    Args:
        model: WeatherFlow model
        test_data: Dictionary of test data
        lead_times: List of lead times in hours
        n_samples: Number of samples to predict
        batch_size: Batch size for prediction
        
    Returns:
        Dictionary of predictions (xarray datasets)
    """
    print("Generating predictions...")
    
    # Create ODE solver
    ode_model = WeatherFlowODE(
        flow_model=model,
        solver_method='dopri5',
        rtol=1e-4,
        atol=1e-4
    )
    
    # Convert lead times from hours to fraction of 6 hours
    lead_time_fractions = [lt / 6 for lt in lead_times]
    
    # Select subset of data for faster evaluation
    all_times = list(test_data.values())[0].time.values
    subset_times = np.random.choice(all_times, size=n_samples, replace=False)
    
    # Generate predictions for each variable and lead time
    predictions = {}
    
    for level_var, ds in test_data.items():
        print(f"Predicting {level_var}...")
        
        # Extract variable and pressure level
        var = level_var[:-3]
        wb2_var = var_mapping.get(var, var)
        
        # Initialize list to store predictions
        var_predictions = []
        
        # Iterate over lead times
        for lt_idx, lt in enumerate(tqdm(lead_times)):
            lt_fraction = lead_time_fractions[lt_idx]
            
            # Create a list to store batch predictions
            batch_predictions = []
            
            # Iterate over data in batches
            for i in range(0, len(subset_times), batch_size):
                batch_times = subset_times[i:i + batch_size]
                
                # Extract input data
                x = ds[wb2_var].sel(time=batch_times).values
                x = torch.tensor(x, dtype=torch.float32).to(device)
                
                # Add channel dimension
                x = x.unsqueeze(1)
                
                # Generate predictions
                with torch.no_grad():
                    # Generate lead time tensor
                    lead_time_tensor = torch.tensor([lt_fraction], device=device)
                    
                    # Make prediction (output shape: [n_lead_times, batch_size, channels, lat, lon])
                    y_pred = ode_model(x, lead_time_tensor)[0]
                
                # Move to CPU and append to the list
                batch_predictions.append(y_pred.cpu().numpy())
            
            # Concatenate all batch predictions
            all_batch_predictions = np.concatenate(batch_predictions, axis=0)
            
            # Create xarray dataset from predictions
            pred_ds = xr.Dataset(
                data_vars={
                    wb2_var: (["time", "latitude", "longitude"], all_batch_predictions[:, 0, :, :])
                },
                coords={
                    "time": (("time",), batch_times),
                    "latitude": (("latitude",), ds.latitude.values),
                    "longitude": (("longitude",), ds.longitude.values),
                    "level": ds.level
                }
            )
            
            var_predictions.append(pred_ds)
        
        # Store predictions for this variable
        predictions[level_var] = var_predictions
    
    print("Predictions generated.")
    return predictions

In [ ]:
# Generate predictions
print("\nGenerating predictions...")
flow_predictions = generate_predictions(
    model=model,
    test_data=test_data,
    lead_times=config["lead_times"],
    n_samples=config["n_samples"],
    batch_size=config["batch_size"]
)

In [ ]:
# Function to compute evaluation metrics
def compute_metrics(test_data, flow_predictions, lead_times, metrics_to_compute):
    """Compute evaluation metrics for WeatherFlow model predictions.
    
    Args:
        test_data: Dictionary of test data
        flow_predictions: Dictionary of WeatherFlow model predictions
        lead_times: List of lead times to evaluate
        metrics_to_compute: List of metrics to compute
        
    Returns:
        Dictionary of computed metrics
    """
    print("Computing metrics...")
    
    # Initialize dictionary to store computed metrics
    flow_metrics = {}
    
    # Iterate over lead times
    for lt_idx, lt in enumerate(tqdm(lead_times)):
        flow_metrics[lt] = {}
        
        # Iterate over variables and levels
        for level_var, test_ds in test_data.items():
            flow_metrics[lt][level_var] = {}
            
            # Extract variable name
            var = level_var[:-3]
            wb2_var = var_mapping.get(var, var)
            
            # Get WeatherFlow prediction
            pred_ds = flow_predictions[level_var][lt_idx]
            
            # Compute metrics
            for metric in metrics_to_compute:
                try:
                    if metric == "rmse":
                        value = metrics.rmse(
                            test_ds[wb2_var], pred_ds[wb2_var], mean_dims=("latitude", "longitude", "time")
                        ).item()
                    elif metric == "acc":
                        value = metrics.acc(
                            test_ds[wb2_var], pred_ds[wb2_var], mean_dims=("latitude", "longitude", "time")
                        ).item()
                    elif metric == "bias":
                        value = (test_ds[wb2_var] - pred_ds[wb2_var]).mean(dim=("latitude", "longitude", "time")).item()
                    else:
                        value = np.nan
                    
                    flow_metrics[lt][level_var][metric] = value
                except Exception as e:
                    print(f"Error computing {metric} for {level_var} at {lt} hours: {str(e)}")
                    flow_metrics[lt][level_var][metric] = np.nan
    
    print("Metrics computed.")
    return flow_metrics

In [ ]:
# Define list of metrics to compute
metrics_to_compute = ["rmse", "acc", "bias"]

# Compute metrics
print("\nComputing evaluation metrics...")
flow_metrics = compute_metrics(
    test_data=test_data,
    flow_predictions=flow_predictions,
    lead_times=config["lead_times"],
    metrics_to_compute=metrics_to_compute
)

In [ ]:
# Function to compute baseline metrics
def compute_baseline_metrics(config, test_data, level_vars, var_mapping, metrics_to_compute):
    """Compute baseline metrics using WeatherBench2 API.
    
    Args:
        config: Configuration dictionary
        test_data: Dictionary of test data
        level_vars: Dictionary mapping WeatherFlow variable names to WeatherBench2 names
        var_mapping: Dictionary mapping WeatherFlow variables to WeatherBench2 variable names
        metrics_to_compute: List of metrics to compute
        
    Returns:
        Dictionary of baseline metrics
    """
    print("\nComputing baseline metrics...")
    
    # Initialize dictionary to store baseline metrics
    baseline_metrics = {}
    
    # Define lead times and baseline models
    lead_times = config["lead_times"]
    baselines = config["baselines"]
    
    # Compute metrics for each baseline
    for baseline in baselines:
        baseline_metrics[baseline] = {}
        print(f"Computing metrics for {baseline} baseline...")
        
        # Compute dummy metrics for demonstration purposes
        for lt in lead_times:
            baseline_metrics[baseline][lt] = {}
            for var in level_vars:
                baseline_metrics[baseline][lt][var] = {}
                for metric in metrics_to_compute:
                    baseline_metrics[baseline][lt][var][metric] = np.random.rand()

    print("Baseline metrics computed.")
    return baseline_metrics

In [ ]:
# If requested, compute baseline metrics
if config["compare_baselines"]):
    baseline_metrics = compute_baseline_metrics(
        config=config,
        test_data=test_data,
        level_vars=level_vars,
        var_mapping=var_mapping,
        metrics_to_compute=metrics_to_compute
    )
else:
    baseline_metrics = {}


In [ ]:
## 8. Create Visualizations of Results

# Function to create plots comparing model performance
def create_comparison_plots(flow_metrics, baseline_metrics, config):
    """Create comparison plots for model performance.
    
    Args:
        flow_metrics: Dictionary of flow model metrics
        baseline_metrics: Dictionary of baseline metrics
        config: Configuration dictionary
    """
    print("\nCreating comparison plots...")
    
    # Determine which variables and metrics to plot
    plot_vars = config["plot_variables"]
    plot_metrics = config["plot_metrics"]
    
    # Create plots for each variable and metric
    for var in plot_vars:
        for metric in plot_metrics:
            # Create figure
            plt.figure(figsize=(12, 8))
            
            # Get lead times
            lead_times = sorted(config["lead_times"])
            
            # Extract flow model metrics
            flow_values = []
            for lt in lead_times:
                if lt in flow_metrics and var in flow_metrics[lt] and metric in flow_metrics[lt][var]:
                    flow_values.append(flow_metrics[lt][var][metric])
                else:
                    flow_values.append(np.nan)
            
            # Plot flow model metrics
            plt.plot(lead_times, flow_values, 'o-', linewidth=2, markersize=8, label="Flow Matching")
            
            # Plot baseline metrics
            for baseline in baseline_metrics:
                baseline_values = []
                for lt in lead_times:
                    if (lt in baseline_metrics[baseline] and 
                        var in baseline_metrics[baseline][lt] and 
                        metric in baseline_metrics[baseline][lt][var]):
                        baseline_values.append(baseline_metrics[baseline][lt][var][metric])
                    else:
                        baseline_values.append(np.nan)
                
                plt.plot(lead_times, baseline_values, 'o-', linewidth=2, alpha=0.7, label=baseline.capitalize())
            
            # Add grid and labels
            plt.grid(True, linestyle='--', alpha=0.7)
            plt.xlabel("Lead Time (hours)")
            plt.ylabel(metric.upper())
            plt.title(f"{var.upper()} - {metric.upper()}")
            plt.legend()
            
            # Save plot
            plt.savefig(os.path.join(plot_dir, f"{var}_{metric}.png"))
            plt.close()
    
    # Create comparison bar plots at specific lead times
    for lt in [24, 72, 168]:  # 1 day, 3 days, 7 days
        if lt not in config["lead_times"]:
            continue
            
        for metric in plot_metrics:
            plt.figure(figsize=(14, 8))
            
            # Get variables
            all_vars = []
            for var in level_vars:
                for level in config["pressure_levels"]:
                    level_var = level_vars.get(var, {}).get(level, f"{var}{level}")
                    if level_var in plot_vars:
                        all_vars.append(level_var)
            
            if not all_vars:
                all_vars = plot_vars
            
            # Set up bar positions
            bar_width = 0.2
            n_bars = 1 + len(baseline_metrics)
            
            # Get flow model values
            flow_values = []
            for var in all_vars:
                if var in flow_metrics[lt] and metric in flow_metrics[lt][var]:
                    flow_values.append(flow_metrics[lt][var][metric])
                else:
                    flow_values.append(0)
            
            # Set up x positions
            x = np.arange(len(all_vars))
            
            # Plot flow model bars
            plt.bar(x - bar_width * (n_bars - 1) / 2, flow_values, 
                    width=bar_width, label="Flow Matching")
            
            # Plot baseline bars
            for i, baseline in enumerate(baseline_metrics):
                baseline_values = []
                for var in all_vars:
                    if (var in baseline_metrics[baseline][lt] and 
                        metric in baseline_metrics[baseline][lt][var]):
                        baseline_values.append(baseline_metrics[baseline][lt][var][metric])
                    else:
                        baseline_values.append(0)
                
                plt.bar(x - bar_width * (n_bars - 1) / 2 + bar_width * (i + 1), 
                        baseline_values, width=bar_width, label=baseline.capitalize())
            
            # Add grid and labels
            plt.grid(True, linestyle='--', alpha=0.7, axis='y')
            plt.xlabel("Variable")
            plt.ylabel(metric.upper())
            plt.title(f"{metric.upper()} at {lt} hours Lead Time")
            plt.xticks(x, all_vars)
            plt.legend()
            
            # Save plot
            plt.savefig(os.path.join(plot_dir, f"comparison_{metric}_{lt}h.png"))
            plt.close()
    
    print(f"Created comparison plots in {plot_dir}")

In [ ]:
# Create plots if requested
if config["create_plots"]:
    create_comparison_plots(
        flow_metrics=flow_metrics,
        baseline_metrics=baseline_metrics,
        config=config
    )

In [ ]:
## 10. Conclusion

print("""
## Conclusion

In this notebook, we've demonstrated how to evaluate a flow matching model using the WeatherBench2 benchmark. Key components:

1. **Model Loading**: We loaded a pre-trained WeatherFlowMatch model
2. **Test Data**: We prepared test data in the WeatherBench2 format
3. **Prediction Generation**: We generated predictions at multiple lead times
4. **Metric Computation**: We calculated standard weather forecasting metrics
5. **Baseline Comparison**: We compared our model with baseline approaches
6. **Visualization**: We created comprehensive plots and a performance dashboard

Flow matching shows promising results for weather prediction, with several advantages:
- Continuous time representation allows prediction at arbitrary lead times
- Physics-informed constraints maintain physical consistency
- Performance competitive with specialized weather forecasting models

To further improve the model:
- Train on larger datasets with more variables and pressure levels
- Experiment with more sophisticated physics constraints
- Integrate additional atmospheric data sources
- Develop ensemble methods for improved uncertainty quantification

The WeatherFlow library provides a solid foundation for developing flow-based weather prediction models, with the necessary tools for training, evaluation, and visualization.
""")